In [1]:
import pandas as pd;
import numpy as np;
from datetime import datetime
from datetime import date
from datetime import timedelta
import calendar
import json


SUPABASE_URL = "Paste your Supabase url here"
SUPABASE_KEY = "Paste your Supabase key here"

from supabase import create_client, Client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
holidays = np.array(['2023-01-26', '2023-03-07', '2023-03-30', '2023-04-04', '2023-04-07', '2023-04-14', '2023-05-01', '2023-06-28', '2023-08-15', '2023-09-19', '2023-10-02', '2023-10-24', '2023-11-14', '2023-11-27', '2023-12-25'])




def get_previous_weekday_date(date):
    one_day = timedelta(days=1)
    while date.weekday() > 4:  # if the given date is a weekend
        date -= one_day
    if date.weekday() == 0:  # if the given date is a Monday
        date -= 3 * one_day
    else:
        date -= one_day
    
    str_date = date.isoformat()
    if str_date in holidays:
        date = get_previous_weekday_date(date)
    return date




now = datetime.now()
today = date.today()

currentMonth = now.strftime("%b").upper()
currentYear = str(now.year)
str_today = today.isoformat()
if str_today in holidays:
    today = get_previous_weekday_date(today)

previous_day = get_previous_weekday_date(today)




# pd.set_option('display.max_rows', None)

def parse_csv_and_save():
    
    current_date = get_previous_weekday_date(today)
    if(now.hour > 18 and  now.hour < 23):
        current_date = today
    

    formatted_date = current_date.strftime("%d%b%Y").upper()
    print('Downloading data for : ' + formatted_date)
    print()

    data_url = 'https://archives.nseindia.com/content/historical/DERIVATIVES/' + currentYear + '/' + currentMonth + '/fo'+  formatted_date + 'bhav.csv.zip'

    df = pd.read_csv(data_url, compression='zip')
    column_to_exclude = ['OPTSTK', 'OPTIDX']


    print("Reading data....")
    filtered_df = df[~df['INSTRUMENT'].isin(column_to_exclude)]

    current_expiry = [filtered_df['EXPIRY_DT'].iloc[0]]


    sub_total_df = filtered_df.groupby('SYMBOL')[['CONTRACTS', 'OPEN_INT', 'CHG_IN_OI']].sum();



    current_expiry_df = filtered_df[filtered_df['EXPIRY_DT'].isin(current_expiry)]
    current_expiry_df.drop(['VAL_INLAKH','SETTLE_PR','LOW','HIGH','OPEN','INSTRUMENT', 'INSTRUMENT', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'CONTRACTS', 'VAL_INLAKH', 'OPEN_INT', 'CHG_IN_OI'], axis=1, inplace=True)

    merged_df = pd.merge(current_expiry_df, sub_total_df, on='SYMBOL')
    merged_df.rename(columns={'SYMBOL': 'symbol'}, inplace=True)
    merged_df.rename(columns={'CLOSE': 'close'}, inplace=True)
    merged_df.rename(columns={'TIMESTAMP': 'timestamp'}, inplace=True)
    merged_df.rename(columns={'CONTRACTS': 'lotsTraded'}, inplace=True)
    merged_df.rename(columns={'OPEN_INT': 'openInterest'}, inplace=True)
    merged_df.rename(columns={'CHG_IN_OI': 'changeOi'}, inplace=True)
    


    merged_df.dropna(axis=1, inplace=True)
    merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], dayfirst = True)  
    merged_df['timestamp'] = merged_df['timestamp'].dt.strftime('%Y-%m-%d')

    

    result = merged_df.to_dict(orient='records')
    result_json = json.dumps(result)
    final_json = json.loads(result_json)

    print()
    print("Inserting in DB....")

    json_obj = {}
    json_obj["data"] = final_json
    json_obj["timestamp"] = merged_df['timestamp'].iloc[0]
    # last_row_data = supabase.table('bhavcopy').select("*").order('id', desc=True).limit(1).single().execute()
    # print(last_row_data.data.timestamp)
    data = supabase.table("bhavcopy").insert(json_obj).execute();
    print(data)
    print(previous_day)
    get_previous_day_data(previous_day)



def format_number(amount):
    def truncate_float(number, places):
        return int(number * (10 ** places)) / 10 ** places

    if amount < 1e3:
        return str(amount)

    if 1e3 <= amount < 1e5:
        return str(truncate_float((amount / 1e5) * 100, 2)) + " K"

    if 1e5 <= amount < 1e7:
        return str(truncate_float((amount / 1e7) * 100, 2)) + " L"

    if amount > 1e7:
        return str(truncate_float(amount / 1e7, 2)) + " Cr"


def last_friday_of_month(year, month):
    last_day = max(calendar.monthcalendar(year, month), key=lambda week: week[calendar.FRIDAY])
    return date(year, month, last_day[calendar.FRIDAY])





def get_percent_change(previous, current):
    percent = (current - previous)/current * 100
    return round(percent, 1)



def map_percent_change_with_todays_data(oldData, newData):
    for index, data in enumerate(newData):
        if oldData[index]['symbol'] == data['symbol']:
            data['oneDayPriceChange'] = get_percent_change(oldData[index]['close'], data['close'])
            data['oneDayOiChange'] = get_percent_change(oldData[index]['openInterest'], data['openInterest'])
            data['openInterest'] = format_number(data['openInterest'])
            data['lotsTraded'] = format_number(data['lotsTraded'])
            data['changeOi'] = format_number(data['changeOi'])
            add_data_in_db(data)




def get_todays_data(date):
    global todays_data
    response = supabase.table('bhavcopy').select("*").eq('timestamp', date).single().execute()
    todays_data = response.data['data']
    map_percent_change_with_todays_data(previous_day_data, todays_data)



def get_previous_day_data(date):
    global previous_day_data
    response = supabase.table('bhavcopy').select("*").eq('timestamp', date).single().execute()
    previous_day_data = response.data['data']
    today = date.today()
    get_todays_data(today)



def get_first_day_data():
    year = now.year
    month = now.month - 1 if now.month > 1 else 12
    expiry_start_date = last_friday_of_month(year, month)
    global expiry_start_data
    response = supabase.table('bhavcopy').select("*").eq('timestamp', expiry_start_date).single().execute()
    expiry_start_data = response.data['data']




def add_data_in_db(data):
    result_json = json.dumps(data)
    final_json = json.loads(result_json)
    response = supabase.table("openInterest").insert(final_json).execute()
    
    print(response)


parse_csv_and_save()










Reading data....

Inserting in DB....


/tmp/ipykernel_10379/3962966080.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_expiry_df.drop(['VAL_INLAKH','SETTLE_PR','LOW','HIGH','OPEN','INSTRUMENT', 'INSTRUMENT', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'CONTRACTS', 'VAL_INLAKH', 'OPEN_INT', 'CHG_IN_OI'], axis=1, inplace=True)


data=[{'id': 43, 'timestamp': '2023-05-31T00:00:00', 'data': [{'close': 44241.65, 'symbol': 'BANKNIFTY', 'changeOi': 7040, 'timestamp': '2023-05-31', 'lotsTraded': 136386, 'openInterest': 2364060}, {'close': 18629.55, 'symbol': 'NIFTY', 'changeOi': -558600, 'timestamp': '2023-05-31', 'lotsTraded': 140051, 'openInterest': 10190200}, {'close': 516.85, 'symbol': 'AARTIIND', 'changeOi': 75000, 'timestamp': '2023-05-31', 'lotsTraded': 1486, 'openInterest': 5251350}, {'close': 4084.2, 'symbol': 'ABB', 'changeOi': 225750, 'timestamp': '2023-05-31', 'lotsTraded': 4322, 'openInterest': 1569250}, {'close': 21917.9, 'symbol': 'ABBOTINDIA', 'changeOi': 640, 'timestamp': '2023-05-31', 'lotsTraded': 362, 'openInterest': 66000}, {'close': 171.75, 'symbol': 'ABCAPITAL', 'changeOi': 1328400, 'timestamp': '2023-05-31', 'lotsTraded': 1775, 'openInterest': 29062800}, {'close': 202.5, 'symbol': 'ABFRL', 'changeOi': -137800, 'timestamp': '2023-05-31', 'lotsTraded': 3116, 'openInterest': 28490800}, {'close':